In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import ccxt
from datetime import datetime, timedelta

In [28]:
# Inicializar o objeto da exchange (Binance neste exemplo)
exchange = ccxt.binance()

# Definir o par de negociação e o intervalo de tempo
symbol = ["ETH","BTC"]#,"FLOKI","WIF","REI","SOL","FRONT","ID","RAY","NEAR","LPT","OM","RNDR","VIDT","PEOPLE","GAS","APT","JTO","VANRY","ZRX","BURGER","ORN","LRC","WAVES","RSR","HIFI"]  # ou 'BTC/USD' dependendo da exchange
timeframe = '1d'  # Pode ser '1m', '5m', '1h', '1d', etc.

In [29]:
symbol = [i + '/USDT' for i in symbol]

In [30]:
#Variáveis do modelo
buy_or_sell = 'buy' #'buy' or 'sell' setup 
data_type = 'adj_close' #'close' or 'adj_close'
# start_date = '2010-02-01' #start date of backtest
# start_date = datetime.strptime(start_date, '%Y-%m-%d')
# start_date = start_date - pd.tseries.offsets.BDay(80)
eden_type = 'my' # 'my' == mm8 > mm80 // 'asc_mm' == mm 8 and 80 ascending // 'all' == both conditions
is_eden = 'sim' #'sim' ou 'nao'
target_type = 'fixed' # 'amp' or 'fixed'
multiplier = 1.61 # if target_type = 'fixed', Multiply the risk to calculate the target. Normally 1.61 or 2

# Funções Auxiliares

### Definindo o algoritmo para simular as operações

In [32]:
import math

# Create a function to round any number to the smalles multiple of 100
def round_down(x):
    return int(math.floor(x / 100.0)) * 100

def backtest_algorithm(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False

    for i in range(0,len(df)):

        if ongoing == True:

            if (df["Open"][i] >= target) | (df["Open"][i] <= stop): 
                exit = df["Open"][i]

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= stop: 
                exit = stop

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["High"][i] >= target: 
                exit = target

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

        else:
            if ~(np.isnan(df["buy_price"][i])):
                entry = df["buy_price"][i]
                stop = df["stop"][i]
                
                if df["Low"][i] > stop: 
                    ongoing = True
                    risk = entry - stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

### Calculando a estatística e a curva de capital

In [33]:
def get_drawdown(data, column = "Close"):
    data["Max"] = data[column].cummax()
    data["Delta"] = data['Max'] - data[column]
    data["Drawdown"] = 100 * (data["Delta"] / data["Max"])
    max_drawdown = data["Drawdown"].max()
    return max_drawdown

def strategy_test(all_profits, total_capital):
    if len(all_profits) > 0:
        gains = sum(x >= 0 for x in all_profits)
        losses = sum(x < 0 for x in all_profits)
        num_operations = gains + losses
        pct_gains = 100 * (gains / num_operations)
        pct_losses = 100 - pct_gains
        total_profit = sum(all_profits)
        pct_profit = (total_profit / total_capital[0]) * 100
        
        # Compute drawdown
        total_capital = pd.DataFrame(data=total_capital, columns=["total_capital"])
        drawdown = get_drawdown(data=total_capital, column="total_capital")

        # Compute profit per operation
        profit_per_operation = pct_profit / num_operations

        # Expeceted Value
        all_positives = [x for x in all_profits if x >= 0]
        if len(all_positives) > 0:
            average_gain = sum(all_positives) / len(all_positives)
        else:
            average_gain = 0

        all_negatives = [x for x in all_profits if x < 0]
        if len(all_negatives) > 0:
            average_loss = sum(all_negatives) / len(all_negatives)
        else:
            average_loss = 0
            
        num_operations = len(all_profits)
        pct_gains = (len(all_positives) / num_operations)
        pct_losses = 1 - pct_gains
        
        expected_value = (average_gain * pct_gains) + (average_loss * pct_losses)
    else:
        total_profit = 0
        pct_profit = 0
        drawdown = 0
        num_operations = 0
        profit_per_operation = 0
        gains = 0
        pct_gains = 0
        losses = 0
        pct_losses = 0
        expected_value = 0

    return {
        "total_profit": total_profit,
        "pct_profit": pct_profit,
        "pct_drawdown": drawdown,
        "num_operations": num_operations,
        "pct_profit_per_operation": profit_per_operation,
        "num_gains": gains ,
        "pct_gains": pct_gains,
        "num_losses": losses,
        "pct_losses": pct_losses, 
        "expected_value": expected_value
    }
def capital_plot(total_capital, all_profits):
  all_profits = [0] + all_profits # make sure both lists are the same size
  cap_evolution = pd.DataFrame({'Capital': total_capital, 'Profit': all_profits})
  plt.title("Curva de Capital")
  plt.xlabel("Total Operações")
  cap_evolution['Capital'].plot()

# Ajuste do DF para chamada das Funções de modo Escalável

In [34]:
if buy_or_sell == 'buy':
    df_stats = pd.DataFrame()
    for stock in symbol:
        # Obter dados OHLC
        ohlcv = exchange.fetch_ohlcv(stock, timeframe)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp',inplace=True)

    ### Definindo o candle sinal
        condition_1 = df["Low"] > df["Low"].shift(1)
        condition_2 = df["Low"].shift(1) < df["Low"].shift(2)
        df["signal"] = condition_1 & condition_2

    ### Calculando o Éden dos Traders
        df["mme8"] = df["Close"].rolling(window=8).mean()
        df["mme80"] = df["Close"].rolling(window=80).mean()

        if eden_type == 'my':
            df["eden"] = (df["mme8"] > df["mme80"])
        elif eden_type == 'asc_mms':
            df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1))
        else:
            df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1)) & (df["mme8"] > df["mme80"])
        df.dropna(inplace=True)

    ### Definindo o preço de compra
        condition_1 = df["signal"].shift(1) == True
        condition_2 = df["eden"].shift(1) == True 
        condition_3 = df["High"] > df["High"].shift(1)
        tick = 0.01

        if is_eden == 'sim':
            df["buy_price"] = np.where(
            condition_1 & condition_2 & condition_3, 
            np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
            np.nan)
        else:
            df["buy_price"] = np.where(
            condition_1 & condition_3, 
            np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
            np.nan)
        
    ### Definindo o alvo
        max_high = df["High"].rolling(3).max()
        min_low = df["Low"].rolling(3).min()
        amplitude = (max_high.shift(1) - min_low.shift(1))
        fixed_risk = ((df["High"].shift(1) +tick) - (df["Low"].shift(2) - tick)) * multiplier
        entry = (df["High"].shift(1)) + tick 
        if target_type == 'fixed':
            df["target"] = fixed_risk + entry
        else:
            df["target"] =  amplitude + entry

    ### Definindo o stop
        df["stop"] = df["Low"].shift(2) - tick

    ### Realizando o Backtest
        all_profits, total_capital = backtest_algorithm(
        df=df,
        capital_exposure=1000,
        initial_capital=100000)
        statistics = strategy_test(all_profits, total_capital)
        statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
        df_stats[stock] = statistics
    df_stats = df_stats.T

In [36]:
df

,Open,High,Low,Close,Volume,signal,mme8,mme80,eden,buy_price,target,stop
timestamp,,,,,,,,,,,,
2023-01-20,21071.59,22755.93,20861.28,22667.21,338079.136590,False,21061.64500,17591.837375,True,NaN,NaN,NaN
2023-01-21,22666.00,23371.80,22422.00,22783.55,346445.484320,False,21418.33750,17624.733750,True,NaN,NaN,NaN
2023-01-22,22783.35,23078.71,22292.37,22707.88,253577.752860,False,21637.45750,17655.984500,True,NaN,27413.7794,20861.27
2023-01-23,22706.02,23180.00,22500.00,22916.45,293588.379380,True,21893.07625,17678.083625,True,NaN,24136.0553,22421.99
2023-01-24,22917.81,23162.20,22462.93,22632.89,293158.782540,False,22073.98125,17694.752625,True,NaN,24609.1265,22292.36
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11,68955.88,72800.00,67024.96,72078.10,75292.825726,False,67792.29625,48645.792250,True,NaN,73150.3333,67861.09
2024-03-12,72078.10,73000.00,68620.82,71452.01,68783.546691,True,68193.08375,48992.665375,True,NaN,80375.4947,68094.74
2024-03-13,71452.00,73650.25,71333.31,73072.41,52659.711647,False,69361.63375,49368.676750,True,73000.01,82619.8566,67024.95


In [35]:
df_stats.sort_values('total_profit')

,total_profit,pct_profit,pct_drawdown,num_operations,pct_profit_per_operation,num_gains,pct_gains,num_losses,pct_losses,expected_value
ETH/USDT,0.0,0.0,0.0,27.0,0.0,27.0,1.0,0.0,0.0,0.0
BTC/USDT,0.0,0.0,0.0,27.0,0.0,27.0,1.0,0.0,0.0,0.0
